# Single Layer Perceptron

In [ ]:
import numpy as np
import pandas as pd

# Perceptron for logical AND function
def logical_and(x1, x2):
    w1, w2, theta = 0.5, 0.5, 0.7
    weighted_sum = x1 * w1 + x2 * w2
    if weighted_sum <= theta:
        return 0
    else:
        return 1

# Perceptron for logical OR function
def logical_or(x1, x2):
    w1, w2, theta = 0.3, 0.3, 0.2
    weighted_sum = x1 * w1 + x2 * w2
    if weighted_sum <= theta:
        return 0
    else:
        return 1

# Function to predict BMI categories using logical AND function
def predict_bmi_logical_and(height, weight):
    w1, w2, theta = 0.5, 0.5, 0.7  # Thresholds for logical AND
    logical_and_result = logical_and(height, weight)
    if logical_and_result == 1:
        return 4  # Category 4 (Overweight)
    else:
        return 1  # Category 1 (Underweight)

# Function to predict BMI categories using logical OR function
def predict_bmi_logical_or(height, weight):
    w1, w2, theta = 0.3, 0.3, 0.2  # Thresholds for logical OR
    logical_or_result = logical_or(height, weight)
    if logical_or_result == 1:
        return 3  # Category 3 (Normal weight)
    else:
        return 2  # Category 2 (Obese)

# Training a perceptron with sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def train_perceptron():
    np.random.seed(1)
    inputs = np.array([[0.7, -0.6]])
    outputs = np.array([[1]])

    # Initializing weights and bias
    weights = np.array([[0.5, 0.5]])
    bias = 0.5
    learning_rate = 0.1

    # Training the perceptron
    for _ in range(10000):
        # Forward propagation
        weighted_sum = np.dot(inputs, weights.T) + bias
        activated_output = sigmoid(weighted_sum)

        # Calculating error
        error = outputs - activated_output

        # Backpropagation using the delta rule
        adjustment = learning_rate * np.dot(inputs.T, error * sigmoid_derivative(activated_output))
        weights += adjustment.T
        bias += np.sum(error)

    return weights, bias

# BMI categories based on BMI value
def get_bmi_category(bmi):
    if bmi < 18.5:
        return "1 Underweight"  # Underweight
    elif 18.5 <= bmi < 25:
        return "2 Normal weight"  # Normal weight
    elif 25 <= bmi < 30:
        return "3 Overweight"  # Overweight
    else:
        return "4 Obese"  # Obese

# Read CSV file using Pandas
data = pd.read_csv('bmi.csv')

# Predicting the output after training
trained_weights, trained_bias = train_perceptron()

# Taking new height and weight inputs
new_height = float(input('Enter height: '))
new_weight = float(input('Enter weight: '))

# Mapping height and weight to inputs for logical functions
logical_and_input = 1 if new_height > 170 and new_weight < 70 else 0
logical_or_input = 1 if new_height > 170 or new_weight < 70 else 0

# Predicting BMI categories using logical functions
predicted_bmi_and = predict_bmi_logical_and(logical_and_input, logical_and_input)
predicted_bmi_or = predict_bmi_logical_or(logical_or_input, logical_or_input)

# Combining the predicted values to calculate BMI and categorize
predicted_bmi = (predicted_bmi_and + predicted_bmi_or) / 2

# Categorizing BMI and printing the category
bmi_category = get_bmi_category(predicted_bmi)
print("Predicted BMI Category:", bmi_category)


Enter height: 165
Enter weight: 80
Predicted BMI Category: 1 Underweight


#Multiple Layer Perceptron

In [15]:
import numpy as np
import pandas as pd

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def initialize_weights_and_bias(input_size, hidden_size, output_size):
    np.random.seed(1)
    weights_input_hidden = np.random.rand(input_size, hidden_size)
    weights_hidden_output = np.random.rand(hidden_size, output_size)
    bias_hidden = np.zeros((1, hidden_size))
    bias_output = np.zeros((1, output_size))
    return weights_input_hidden, weights_hidden_output, bias_hidden, bias_output

def forward_propagation(inputs, weights_input_hidden, weights_hidden_output, bias_hidden, bias_output):
    hidden_output = sigmoid(np.dot(inputs, weights_input_hidden) + bias_hidden)
    predicted_output = sigmoid(np.dot(hidden_output, weights_hidden_output) + bias_output)
    return hidden_output, predicted_output

def backpropagation(inputs, targets, hidden_output, predicted_output,
                    weights_hidden_output, weights_input_hidden, bias_hidden, bias_output, learning_rate):
    # Calculate the error e = Yt - YD
    error = targets - predicted_output
    # Calculate the gradients for the output layer
    output_delta = error * predicted_output * (1 - predicted_output)
    # Update weights and biases for the output layer (Wnew)
    weights_hidden_output += learning_rate * np.dot(hidden_output.T, output_delta)
    bias_output += learning_rate * np.sum(output_delta, axis=0, keepdims=True)

    # Calculate the gradients for the hidden layer
    hidden_delta = np.dot(output_delta, weights_hidden_output.T) * hidden_output * (1 - hidden_output)
    # Update weights and biases for the hidden layer
    weights_input_hidden += learning_rate * np.dot(inputs.T, hidden_delta)
    bias_hidden += learning_rate * np.sum(hidden_delta, axis=0, keepdims=True)

    return weights_hidden_output, weights_input_hidden, bias_hidden, bias_output


def train_neural_network(inputs, targets, hidden_size, output_size, epochs, learning_rate):
    input_size = inputs.shape[1]

    weights_input_hidden, weights_hidden_output, bias_hidden, bias_output = initialize_weights_and_bias(
        input_size, hidden_size, output_size)

    for epoch in range(epochs):
        # Forward Propagation
        hidden_output, predicted_output = forward_propagation(
            inputs, weights_input_hidden, weights_hidden_output, bias_hidden, bias_output)

        # Backpropagation
        weights_hidden_output, weights_input_hidden, bias_hidden, bias_output = backpropagation(
            inputs, targets, hidden_output, predicted_output,
            weights_hidden_output, weights_input_hidden, bias_hidden, bias_output, learning_rate)

    return weights_input_hidden, weights_hidden_output, bias_hidden, bias_output


def predict_bmi_category(inputs, weights_input_hidden, weights_hidden_output, bias_hidden, bias_output):
    _, predicted_output = forward_propagation(
        inputs, weights_input_hidden, weights_hidden_output, bias_hidden, bias_output)

    return predicted_output


data = pd.read_csv('bmi.csv')
features = data[['Height', 'Weight']].values
labels = data['Index'].values.reshape(-1, 1)

# Normalize features
features = (features - features.mean(axis=0)) / features.std(axis=0)
# Combine features and labels
data_normalized = np.hstack((features, labels))
# Separate inputs and targets
inputs = data_normalized[:, :-1]
targets = data_normalized[:, -1].reshape(-1, 1)  # Reshape targets to have the same shape as predictions

# Train the neural network
hidden_size = 3
output_size = 1
epochs = 10000
learning_rate = 0.1

weights_input_hidden, weights_hidden_output, bias_hidden, bias_output = train_neural_network(
    inputs, targets, hidden_size, output_size, epochs, learning_rate)

# new input
new_height = 179
new_weight = 70

# Normalize new inputs
new_inputs = (np.array([[new_height, new_weight]]) - features.mean(axis=0)) / features.std(axis=0)

# Predicting BMI category using the trained neural network
predicted_bmi_category = predict_bmi_category(
    new_inputs, weights_input_hidden, weights_hidden_output, bias_hidden, bias_output)

# Convert predicted category to BMI
predicted_bmi = int(predicted_bmi_category.round())
print("Predicted BMI Category:", predicted_bmi)


Predicted BMI Category: 1


<ipython-input-15-7e08af7285c3>:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
